In [ ]:
def main():
    env = gym.make('CartPole-v1')
    model = RECURRENT_PPO()
    score = 0.0
    print_interval = 20

    for n_epi in range(800):
        h_out = (torch.zeros([1, 1, 32], dtype=torch.float), torch.zeros([1, 1, 32], dtype=torch.float))
        s = env.reset()
        done = False

        prob_pre_action_0 = 0.5
        prob_pre_action_1 = 0.5

        while not done:
            for t in range(T_horizon):
                h_in = h_out
                
                prob, h_out = model.pi(torch.from_numpy(s).float(), h_in, prob_pre_action_0, prob_pre_action_1)
                
                prob = prob.view(-1)
                
                m = Categorical(prob)
                
                a = m.sample().item()
                s_prime, r, done = env.step(a)

                prob_pre_action = torch.tensor(prob)

                model.put_data((s, a, r/100.0, s_prime, prob[a].item(), h_in, h_out, done, prob_pre_action_0, prob_pre_action_1))

                s = s_prime
                prob_pre_action_0 = prob[0]
                prob_pre_action_1 = prob[1]


                score += r
                if done:
                    break
            model.train_net()

        if n_epi%print_interval==0 and n_epi!=0:
            print("# of episode :{}, avg score : {:.1f}".format(n_epi, score/print_interval))
            score = 0.0

    env.close()

if __name__ == '__main__':
    main()

end = time.time()

print(f"{end - start:.5f} sec")